# Keyword Frequency Analysis by Category

This notebook analyzes keyword frequencies across different transaction categories to build a comprehensive keyword dictionary. It filters out stopwords and short keywords to identify meaningful terms for each category.

## Processing Steps
1. Load transaction data from input file
2. Normalize all narrations
3. Extract keywords by category
4. Calculate frequency statistics
5. Filter stopwords and numeric terms
6. Export cleaned keyword frequency data

## Output
Generates an Excel file containing:
- Keywords with their frequencies per category
- Transaction count per category
- Overall frequency in complete dataset

In [ ]:
# Import required libraries
import pandas as pd  # Data manipulation and analysis
import numpy as np   # Numerical operations
import re            # Text processing
from collections import Counter  # Count keyword occurrences

In [ ]:
# Define input/output file paths
input_file = "credit_txn_v5.xlsx"
output_file = "keyword_freq_by_category_credit_v5.xlsx"

In [ ]:
# Load transaction data
all_df = pd.read_excel(input_file)

In [ ]:
def normalize(text):
    """Normalize narration text for consistent analysis"""
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [ ]:
# Apply normalization to all narrations
all_df["narr_norm"] = all_df["Narration"].apply(normalize)

In [ ]:
# Extract unique transaction categories
categories = (
    all_df['Category']
    .dropna()
    .unique()
)

In [ ]:
def keyword_freq_df(df, narr_col, category_name, min_freq=10):
    """
    Calculate keyword frequencies for a specific category.
    
    Args:
        df (DataFrame): Transactions for the category
        narr_col (str): Column name containing narrations
        category_name (str): Category label
        min_freq (int): Minimum frequency threshold
        
    Returns:
        DataFrame: Keywords with frequencies and metadata
    """
    # Extract all tokens and count frequencies
    tokens = []
    for n in df[narr_col].dropna():
        tokens.extend(n.split())

    freq = Counter(tokens)
    category_txn_count = len(df)

    # Create dataframe with filtered keywords
    freq_df = (
        pd.DataFrame(freq.items(), columns=['keyword', 'freq'])
          .query('freq > @min_freq')  # Filter low-frequency keywords
          .sort_values('freq', ascending=False)
          .reset_index(drop=True)
    )

    # Add category and transaction count metadata
    freq_df['category'] = category_name
    freq_df['no_of_transactions'] = category_txn_count

    return freq_df

In [ ]:
# Process keywords for each category
all_freq_dfs = []

for category in categories:
    # Filter transactions for current category (case-insensitive)
    ref_df = all_df[
        all_df['Category']
        .str.strip().str.lower()
        == category.strip().lower()
    ]

    if ref_df.empty:
        continue

    # Calculate keyword frequencies for this category
    freq_df = keyword_freq_df(
        ref_df,
        narr_col='narr_norm',
        category_name=category,
        min_freq=10
    )

    all_freq_dfs.append(freq_df)

# Combine all categories and sort
final_df = pd.concat(all_freq_dfs, ignore_index=True)

final_df = (
    final_df
    .sort_values(
        by=['category', 'freq'],
        ascending=[True, False]
    )
    .reset_index(drop=True)
)

In [ ]:
# Calculate total frequency for all keywords in complete dataset
all_tokens = []
for n in all_df['narr_norm'].dropna():
    all_tokens.extend(n.split())

total_freq_counter = Counter(all_tokens)

In [ ]:
# Map total dataset frequency for each keyword
final_df['freq_total_dataset'] = final_df['keyword'].map(
    lambda k: total_freq_counter.get(k, 0)
)

In [ ]:
# Define stopwords and helper functions to filter non-meaningful keywords
STOPWORDS = {
    'in','on','at','for','to','from','by','with','about','against',
    'between','into','through','during','before','after','above','below',
    'of','off','over','under','and','or','the','a','an','is','are',
    'this','that','these','those'
}

def is_numeric_like(word):
    """Check if word contains numeric characters"""
    return any(char.isdigit() for char in str(word))

In [ ]:
# Clean keywords by removing stopwords, numeric-like terms, and very short words
df_clean = (
    final_df.assign(keyword=final_df['keyword'].astype(str).str.lower().str.strip())
      .loc[~final_df['keyword'].apply(is_numeric_like)]  # Remove numeric-like
      .loc[~final_df['keyword'].isin(STOPWORDS)]  # Remove stopwords
      .loc[final_df["keyword"].str.len() > 2]  # Remove very short keywords
      .reset_index(drop=True)
)

In [ ]:
# Export cleaned keyword frequencies to Excel
df_clean.to_excel(
   output_file,
    index=False
)

In [14]:
txn_types = (
    all_df['Debit/Credit']
    .dropna()
    .str.strip()
    .str.lower()
    .unique()
)


In [15]:
def keyword_freq_by_txn_type(df, narr_col, txn_col, txn_type, min_freq=10):
    ref_df = df[
        df[txn_col].str.strip().str.lower() == txn_type
    ]

    if ref_df.empty:
        return None

    tokens = []
    for n in ref_df[narr_col].dropna():
        tokens.extend(n.split())

    freq = Counter(tokens)

    freq_df = (
        pd.DataFrame(freq.items(), columns=['keyword', 'freq'])
          .query('freq > @min_freq')
          .sort_values('freq', ascending=False)
          .reset_index(drop=True)
    )

    freq_df['transaction_type'] = txn_type

    return freq_df


In [16]:
all_tokens = []
for n in all_df['narr_norm'].dropna():
    all_tokens.extend(n.split())

global_freq_counter = Counter(all_tokens)


In [17]:
all_txn_freq_dfs = []

for txn_type in txn_types:
    freq_df = keyword_freq_by_txn_type(
        df=all_df,
        narr_col='narr_norm',
        txn_col='Debit/Credit',
        txn_type=txn_type,
        min_freq=10
    )

    if freq_df is not None:
        all_txn_freq_dfs.append(freq_df)


In [18]:
final_txn_df = (
    pd.concat(all_txn_freq_dfs, ignore_index=True)
      .sort_values(
          by=['transaction_type', 'freq'],
          ascending=[True, False]
      )
      .reset_index(drop=True)
)


In [19]:
final_txn_df['freq_total_dataset'] = final_txn_df['keyword'].map(
    lambda k: global_freq_counter.get(k, 0)
)

In [20]:
credit_df = final_txn_df[
    final_txn_df['transaction_type'] == 'credit'
]

debit_df = final_txn_df[
    final_txn_df['transaction_type'] == 'debit'
]


In [22]:
credit_df = credit_df.drop(columns=['transaction_type'])
debit_df = debit_df.drop(columns=['transaction_type'])


In [23]:
output_file = "keyword_frequency_by_transaction_type.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    credit_df.to_excel(
        writer,
        sheet_name="credit",
        index=False
    )
    debit_df.to_excel(
        writer,
        sheet_name="debit",
        index=False
    )
